In [1]:
print('ok')

ok


In [ ]:
from langchain_community.document_loaders import PyPDFLoader , DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableLambda,RunnableParallel,RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from typing import List
from langchain_core.documents import Document

In [4]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        path = data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [13]:
extracted_data = load_pdf_files('data')

In [28]:
# extracted_data

In [ ]:
def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [17]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [27]:
# minimal_docs

In [23]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=25,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [25]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks:{len(texts_chunk)}")

Number of chunks:5867


In [ ]:
def download_embbeding_model():
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding

In [36]:
embedding = download_embbeding_model()

In [11]:
len(embedding.embed_query("prince"))

384

In [12]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [13]:
## load my api-key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [ ]:
## authentication
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
pc

In [ ]:
from pinecone import ServerlessSpec 

index_name = "medicalchatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [52]:
vectorstore = PineconeVectorStore.from_documents(documents=texts_chunk, index_name='medical-chatbot', embedding=embedding)

In [53]:
vectorstore

>> production

In [ ]:
index_name = "medicalchatbot"

In [15]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [16]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [17]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='78221288-de9f-4724-84ec-0ea04fd5f4ef', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='ad3aaeb0-1126-476a-9ec8-e880de93db51', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='9cbf4058-e439-446d-a523-44538d9fc691', metadata={'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged wi

In [18]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [19]:
format_docs(retrieved_docs)

'GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26\n\nGALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25\n\nAcidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'

In [57]:
for doc in retrieved_docs:
    print(doc.page_content)

GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26
GALE ENCYCLOPEDIA OF MEDICINE 2 25
Acne
Acne vulgaris affecting a woman’s face. Acne is the general
name given to a skin disorder in which the sebaceous
glands become inflamed. (Photograph by Biophoto Associ-
ates, Photo Researchers, Inc. Reproduced by permission.)
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25
Acidosis see Respiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when the
pores of the skin become clogged with oil, dead skin
cells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is
the most common skin disease. It affects nearly 17 million
people in the United States. While acne can arise at any


In [33]:
chatModel = ChatOpenAI(model="gpt-4o")
parser = StrOutputParser()

In [6]:
prompt = PromptTemplate(
    template="""
    Use the following pieces of information to answer the user's question.
    If you don't know the answer, just say that you don't know, didn't try to make up an answer.
    
    Context: {context}
    Question: {question}
    
    Only return the helpful answer below and nothing else.
    Helpful answer:
    """,
    input_variables=["context", "question"]
)

In [23]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [24]:
parallel_chain.invoke('what is acne')

{'context': 'GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26\n\nAcidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any\n\nGALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25',
 'question': 'what is acne'}

In [27]:
main_chain = parallel_chain | prompt | chatModel | parser

In [31]:
main_chain.invoke('what is fever also give me it medicine')

'Fever is an increase in body temperature, often due to an underlying infection. In the context of adenovirus infections in children, it may be accompanied by symptoms like conjunctivitis, sore throat, runny nose, and cervical adenitis. Acetaminophen (Tylenol) or aspirin can be used to relieve the pain, aches, and fever.'